### Imports & Utility Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from queue import PriorityQueue
import csv
import random
import math

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = float('inf')

def get_obstacles(file_path: str):
    """Imports obstacles from csv.

    Args:
      file_path: Path to csv file

    Returns:
      obstacles: List of obstacle (x,y) coordinates 

    """

    obstacles = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            obstacles.append((float(row[0]), float(row[1])))
    return obstacles

def get_distance(p1, p2):
    """Calculates euclidean distance between two points.

    Args:
      p1: (x,y) coordinates of first point
      p2: (x,y) coordinates of second point

    Returns:
      distance: Distance between p1 & p2 
    """

    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def get_closest_node(tree, point):
    """Searches tree and finds the closest node to point.

    Args:
      tree: List of explored nodes
      point: (x,y) coordinates of point

    Returns:
      closest_node: Node that's closest to the (x,y) point
    """

    closest_node = tree[0]
    min_distance = get_distance((closest_node.x, closest_node.y), point)

    for node in tree:
        distance = get_distance((node.x, node.y), point)
        if distance < min_distance:
            closest_node = node
            min_distance = distance

    return closest_node

def is_collision(node, obstacles, obstacle_radius=5):
    """Checks if the node collides with any existing obstacles.

    Args:
      node: Node object
      obstacles: List of all obstacle (x,y) coordinates
      obstacle_radius: Radius of obstacles, used to check for collision

    Returns:
      is_collision: Boolean representing the collision status
    """

    for obs in obstacles:
        if get_distance((node.x, node.y), obs) <= obstacle_radius:
            return True
    return False

def is_line_collision(p1, p2, obstacles, obstacle_radius=5, num_checkpoints=10):
    for i in range(num_checkpoints + 1):
        t = i / num_checkpoints
        x = p1.x + t * (p2.x - p1.x)
        y = p1.y + t * (p2.y - p1.y)
        
        if is_collision(Node(x,y), obstacles, obstacle_radius):
            return True
        
    return False

def extract_path(goal_node):
    """Finds full path to current node.

    Args:
      goal_node: Destination node

    Returns:
      path: Path from root to goal node
    """

    path = []
    node = goal_node

    while node.parent is not None:
        path.append(node)
        node = node.parent

    path.append(node)
    return path[::-1]


def plot_path(tree, path, obstacles, start, goal, radius=5):
    """Visualize the full exploration tree & path.

    Args:
      tree: List of explored nodes.

    Returns:
      path: Path from root to goal node
    """

    plt.figure(figsize=(15,15))
    plt.title("Explored Nodes: " + str(len(tree)) + "\nPath Length: " + str(len(path)))
    for obs in obstacles:
        plt.gca().add_patch(plt.Circle(obs, radius, color='r'))
    plt.plot([node.x for node in tree], [node.y for node in tree], 'o', markersize=5)
    plt.plot([node.x for node in path], [node.y for node in path], 'o', markersize=5)
    plt.plot(start[0], start[1], 'bo', markersize=10, label="Start", alpha=0.3)
    plt.plot(goal[0], goal[1], 'go', markersize=10, label="Goal", alpha=0.3)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()


### Rapidly-exploring Random Tree (RRT) Algorithm Overview

RRT is a path planning algorithm that attempts to find a reasonable path from an inital state to a goal state within an environment.  It does this by iteratively building a tree structure as it randomly explores the environment.  The flow of the algorithm is detailed below.

![RRT Flow Diagram](./rrt.png)

### Rapidly-exploring Random Tree (RRT)

In [ ]:
def rrt(start, goal, obstacles, max_iter=5000, exploit_ratio=5, step_size=1):
    """Implementation of RRT algorithm.

    Args:
      start: (x,y) coordinate of start
      goal: (x,y) coordinate of goal
      obstacles: List of all obstacle (x,y) coordinates
      max_iter: Number of iterations to perform before quitting
      exploit_ratio: Percentage of time the algorithm exploits (0-100)
      step_size: Size of each step

    Returns:
      tree: List of explored nodes
    """

    start_node = Node(start[0], start[1])
    goal_node = Node(goal[0], goal[1])
    tree = [start_node]

    for _ in range(max_iter):
        # Plot random point in grid or plot point at goal (explore/exploit)
        if random.randint(0, 100) > exploit_ratio:
            random_point = (random.uniform(0, 100), random.uniform(0, 100))
        else:
            random_point = goal

        # Find the closest existing node
        closest_node = get_closest_node(tree, random_point)

        # Calculate angle between existing node and point
        heading = math.atan2(random_point[1] - closest_node.y, random_point[0] - closest_node.x)

        # Add new node 1 step away from existing node
        new_node = Node(closest_node.x + step_size * math.cos(heading), closest_node.y + step_size * math.sin(heading))
        new_node.parent = closest_node

        if not is_collision(new_node, obstacles):
            # Add node to tree if no collision exists
            tree.append(new_node)

            # Finish if goal is found
            if get_distance((new_node.x, new_node.y), goal) < step_size:
                goal_node.parent = new_node
                tree.append(goal_node)
                return tree, extract_path(tree[-1])

    # Goal not found
    return None, None

### Run & Visualize RRT

In [ ]:
# Init start, goal, and obstacles.
start = (0, 0)
goal = (100, 100)
obstacle_file = 'obstacles.csv'
obstacles = get_obstacles(obstacle_file)

tree, path = rrt(start, goal, obstacles, exploit_ratio=10, step_size=1, max_iter=5000)
if tree:
    plot_path(tree, path, obstacles, start, goal)
else:
    print("Path not found.")

### Probabilistic Roadmap (PRM) Algorithm Overview

PRM is a path planning algorithm that generates a roadmap within an environment.  It does this by iteratively adding nodes & edges to a graph structure as it randomly explores the environment.  The roadmap represents collision-free pathways an agent can use to navigate the environment.  The flow of the algorithm is detailed below.

![PRM Flow Diagram](./prm.png)

### PRM

In [ ]:
def prm(start, goal, obstacles, num_samples=250, max_neighbors=10, neighbor_radius=15):
    """Implementation of PRM algorithm.

    Args:
      start: (x,y) coordinate of start
      goal: (x,y) coordinate of goal
      obstacles: List of all obstacle (x,y) coordinates
      num_samples: Number of nodes to randomly generate before quitting
      max_neighbors: Maximum amount of edges per node
      neighbor_radius: Max distance for edges

    Returns:
      tree: List of explored nodes
    """

    nodes = [Node(start[0], start[1]), Node(goal[0], goal[1])]
    edges = {}

    for _ in range(num_samples):
        node = Node(random.uniform(0, 100), random.uniform(0, 100))

        if is_collision(node, obstacles):
            continue

        nodes.append(node)

        neighbors = sorted([(get_distance((node.x, node.y), (n.x, n.y)), n) for n in nodes[:-1]])[:max_neighbors]

        for dist, neighbor in neighbors:
            if dist < neighbor_radius and not is_line_collision(node, neighbor, obstacles, obstacle_radius=5):
                if node not in edges:
                    edges[node] = []
                edges[node].append(neighbor)
                if neighbor not in edges:
                    edges[neighbor] = []
                edges[neighbor].append(node)

    return nodes, edges

def a_star(start, goal, edges):
    open_set = PriorityQueue()
    start.cost = 0
    open_set.put((0, start))

    while not open_set.empty():
        _, current = open_set.get()

        if current == goal:
            return

        for neighbor in edges[current]:
            tentative_cost = current.cost + get_distance((current.x, current.y), (neighbor.x, neighbor.y))

            if tentative_cost < neighbor.cost:
                neighbor.parent = current
                neighbor.cost = tentative_cost
                estimated_total_cost = tentative_cost + get_distance((neighbor.x, neighbor.y), (goal.x, goal.y))
                open_set.put((estimated_total_cost, neighbor))

In [ ]:
def plot_prm(nodes, edges, start, goal, obstacles, radius, path):

    plt.figure(figsize=(15,15))

    for node, neighbors in edges.items():
        for neighbor in neighbors:
            plt.plot([node.x, neighbor.x], [node.y, neighbor.y], 'C0-', alpha=0.2)

    for obs in obstacles:
        plt.gca().add_patch(plt.Circle(obs, radius, color='r'))

    if path:
        plt.plot([node.x for node in path], [node.y for node in path], 'C1o-', markersize=5)

    plt.plot(start[0], start[1], 'bo', markersize=10, label="Start", alpha=0.3)
    plt.plot(goal[0], goal[1], 'go', markersize=10, label="Goal", alpha=0.3)

    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()


start = (0, 0)
goal = (100, 100)
obstacle_radius = 5
obstacle_file = 'obstacles.csv'
obstacles = get_obstacles(obstacle_file)

nodes, edges = prm(start, goal, obstacles,num_samples=200)
start_node, goal_node = nodes[0], nodes[1]
a_star(start_node, goal_node, edges)
path = extract_path(goal_node)
plot_prm(nodes, edges, start, goal, obstacles, obstacle_radius, path)


### Kalman Filter Overview

The Kalman Filter is an estimation algorithm used to blend a prediction of the system state with the actual observation/measurements of the system state.  It does this by calculating the **Kalman Gain**, which is used to weight the confidence of the observed state vs the predicted state depending on external factors such as noise.  The flow of the algorithm is detailed as follows.

![Kalman Flow Diagram](./kalman.png)

### Particle Filter Overview

The Particle Filter is an estimation algorithm used to estimate the system state in non-linear and non-Gaussian spaces.  It does this by calculating the mean of the most likely guesses of an observed state. The flow of the algorithm is detailed as follows.

![Particle Flow Diagram](./particle.png)